In [8]:
from imutils.perspective import four_point_transform as FPT
from collections import Counter
import matplotlib.pyplot as plt
from imutils import contours
from skimage import io
import numpy as np
import imutils
import cv2
import re

import warnings

warnings.filterwarnings("ignore")


In [9]:
def get_image(url, iter = 2, width = None):
    image = io.imread(url)
    if width:
        image = imutils.resize(image, width)
    ans = image.copy()
    accumEdged = np.zeros(image.shape[:2], dtype="uint8")
    # convert image to black and white
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # blur to remove some of the noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # get edges
    edged = cv2.Canny(blurred, 75, 200)
    accumEdged = cv2.bitwise_or(accumEdged, edged)
    # get contours
    ctrs = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    ctrs = imutils.grab_contours(ctrs)
    docCnt = None

    # ensure that at least one contour was found
    if len(ctrs) > 0:
        # sort the contours according to their size in
        # descending order
        ctrs = sorted(ctrs, key=cv2.contourArea, reverse=True)

        # loop over the sorted contours
        for c in ctrs:
            # approximate the contour
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)

            # if our approximated contour has four points,
            # then we can assume we have found the paper
            if len(approx) == 4:
                docCnt = approx
                break

    paper = image.copy()

    # apply Otsu's thresholding method to binarize the image
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    kernel = np.ones((5,5), np.uint8)
    # erode and dilate to remove some of the unnecessary detail
    thresh = cv2.erode(thresh, kernel, iterations = iter)
    thresh = cv2.dilate(thresh, kernel, iterations = iter)

    # find contours in the thresholded image
    ctrs = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    ctrs = imutils.grab_contours(ctrs)

    return image, ctrs, paper, gray, edged, thresh

    # plot image without axes
    def display(img):
    fig = plt.figure(figsize = (8,12))
    plt.imshow(img)
    plt.axis('off')
    plt.show()
